In [1]:
# crystal structure data
# Using ase, read atoms structure
from ase import Atom, Atoms
from ase.io import read, write
from ase.neighborlist import neighbor_list
from ase.visualize.plot import plot_atoms
#from ase_notebook import AseView, ViewConfig, get_example_atoms

palette = ['#43AA8B', '#F8961E', '#F94144']
sub = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

# data pre-processing and visualization
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# utilities
import time
#from mendeleev import element
from tqdm import tqdm

# Add the external folder path to sys.path
import sys
sys.path.append("../utils")
from utils_data import (load_data, plot_example, plot_predictions)
from utils_model import Network, visualize_layers

bar_format = '{l_bar}{bar:10}{r_bar}{bar:-10b}'
default_dtype = torch.float64
torch.set_default_dtype(default_dtype)

# Create a colormap based on the number of unique symbols
datasets = ['g', 'y', 'r']
colors = dict(zip(datasets, palette))
cmap = mpl.colors.LinearSegmentedColormap.from_list('cmap', [palette[k] for k in [0,2,1]])

# Check device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('torch device:' , device)

ModuleNotFoundError: No module named 'utils_data'

In [ ]:
# Path to your CIF file
cif_file_path = './Si-defect.cif'

# Reading the CIF file
struct = read(cif_file_path)

# Extracting atomic positions
positions = struct.get_positions()

# Extracting the chemical formula
formula = struct.get_chemical_formula()

# Extracting atomic symbols
symbols = struct.get_chemical_symbols()
z = dict(zip(symbols, range(len(symbols))))

# Extracting cell dimensions
cell = struct.get_cell()

# Create the plot
fig, ax = plt.subplots(figsize=(5,5))
norm = plt.Normalize(vmin=0, vmax=len(symbols)-1)
color = [mpl.colors.to_hex(k) for k in cmap(norm([z[j] for j in list(symbols)]))]
ax.set_title(formula)
plot_atoms(struct, ax, radii=0.25, colors=color, rotation=('0x,0y,0z'))

In [ ]:
# crystal structure data
from ase import Atom, Atoms
from ase.io import read, write
from ase.neighborlist import neighbor_list
from ase.visualize.plot import plot_atoms
#from ase_notebook import AseView, ViewConfig, get_example_atoms

palette = ['#43AA8B', '#F8961E', '#F94144']
sub = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

# data pre-processing and visualization
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd


,id,formula,structure,energies,absorption_coefficient,imag_dielectric,real_dielectric,energy_max,bandgap
935,mp-733,GeO2,"(Atom('Ge', [1.3730217033234977, 2.37814306074...","[0.0, 0.0392, 0.0783, 0.1175, 0.1567, 0.1958, ...","[0.0, 0.32852818576627957, 1.3124033386526137,...","[0.0, 0.0003, 0.0006, 0.0009000000000000001, 0...","[3.2907333333333333, 3.290766666666667, 3.2909...",78.3376,3.2348
936,mp-1065514,YTlTe2,"(Atom('Y', [12.051129954999999, 0.0, 1.8952358...","[0.0, 0.0391, 0.0781, 0.1172, 0.1562, 0.1953, ...","[0.0, 3.930355669823682, 15.739175717204319, 3...","[0.0, 0.006433333333333333, 0.0129, 0.0194, 0....","[10.518099999999999, 10.519366666666668, 10.52...",78.1154,0.6338999999999997
937,mp-7439,K3Cu3P2,"(Atom('K', [6.790361937514462, 4.3780980723005...","[0.0, 0.0587, 0.1175, 0.1762, 0.2349, 0.2937, ...","[0.0, 10.698002422282322, 43.080361666969445, ...","[0.0, 0.010366666666666666, 0.0208666666666666...","[8.3065, 8.309533333333334, 8.318600000000002,...",117.4732,1.3053
938,mp-13276,SrLiP,"(Atom('Sr', [0.0, 0.0, 0.0], index=0), Atom('S...","[0.0, 0.0401, 0.0802, 0.1203, 0.1604, 0.2005, ...","[0.0, 3.9082984808102155, 15.675868999546559, ...","[0.0, 0.0057, 0.011433333333333332, 0.01723333...","[8.782833333333333, 8.783966666666666, 8.78736...",80.1856,1.3567000000000005
939,mp-999138,SmTlS2,"(Atom('Sm', [0.0, 0.0, 0.0], index=0), Atom('T...","[0.0, 0.0399, 0.0797, 0.1196, 0.1594, 0.1993, ...","[0.0, 1.8460086114985848, 7.422592382932063, 1...","[0.0, 0.0025333333333333336, 0.0051, 0.0076333...","[7.699466666666666, 7.7, 7.701466666666668, 7....",79.7197,1.7069


In [12]:
import pandas as pd
path = "absorption_mp_data.pkl"
the_data = pd.read_pickle(path)
the_data.head()


,id,formula,structure,energies,absorption_coefficient,imag_dielectric,real_dielectric,energy_max,bandgap
0,mp-546266,DyBi2IO4,"(Atom('Dy', [0.0, 0.0, 0.0], index=0), Atom('B...","[0.0, 0.0277, 0.0554, 0.0831, 0.1109, 0.1386, ...","[0.0, 0.8362304735329499, 3.309178082718349, 7...","[0.0, 0.0015666666666666665, 0.0031, 0.0046666...","[6.9163, 6.916533333333334, 6.9172, 6.9182, 6....",55.4328,1.3818000000000001
1,mp-9583,K2ZnF4,"(Atom('K', [1.2531985199117757, 1.368027219932...","[0.0, 0.0399, 0.0798, 0.1197, 0.1596, 0.1995, ...","[0.0, 0.1362578822994739, 0.4541860898852432, ...","[0.0, 0.0001, 0.00016666666666666666, 0.0002, ...","[2.2021333333333333, 2.202166666666667, 2.2022...",79.7804,4.3866
2,mp-22988,CsGeCl3,"(Atom('Cs', [7.78333021211009, 0.0, 5.48124386...","[0.0, 0.029, 0.058, 0.087, 0.116, 0.1449, 0.17...","[0.0, 0.515935972729374, 2.0145311056971553, 4...","[0.0, 0.0007, 0.0013666666666666669, 0.0020333...","[3.975733333333333, 3.9758333333333336, 3.9761...",57.9759,2.178
3,mp-861502,AcFeO3,"(Atom('Ac', [0.0, 0.0, 0.0], index=0), Atom('F...","[0.0, 0.0587, 0.1174, 0.176, 0.2347, 0.2934, 0...","[0.0, 11295.053197761259, 44864.495977063896, ...","[0.0, 30.1914, 49.9196, 37.98413333333333, 21....","[67.54320000000001, 59.6215, 29.6067, 1.281, -...",117.3625,0.0
4,mp-1025029,PrHSe,"(Atom('Pr', [-2.0265362, -1.17002398670996, -6...","[0.0, 0.0365, 0.073, 0.1095, 0.1459, 0.1824, 0...","[0.0, 1.6849892209331498, 6.697792903781345, 1...","[0.0, 0.0026999999999999997, 0.005366666666666...","[8.784633333333334, 8.7851, 8.786566666666667,...",72.9718,1.9455999999999998


In [34]:
a_structure = the_data["structure"][4]
for i in a_structure:
    #print(type(i))
    print([i.number, i.index, i.position])
    '''
    print(i.position)
    print(i.index)
    print(i.number)'''

[59, 0, array([-2.0265362 , -1.17002399, -6.03032284])]
[59, 1, array([-2.0265362 ,  1.17002399, -2.01010761])]
[1, 2, array([-2.0265362 , -1.17002399, -2.01010761])]
[1, 3, array([-2.0265362 ,  1.17002399, -6.03032284])]
[34, 4, array([0., 0., 0.])]
[34, 5, array([ 0.        ,  0.        , -4.02021523])]


In [ ]:
def numerize_struct(a_structure):
    atom_list = []
    for i in a_structure:
        each_atom = [i.number, i.index, i.position]
        atom_list.append(each_atom)
        
    return(atom_list)
        
print(numerize_struct(a_structure))          
    

[[59, 0, array([-2.0265362 , -1.17002399, -6.03032284])], [59, 1, array([-2.0265362 ,  1.17002399, -2.01010761])], [1, 2, array([-2.0265362 , -1.17002399, -2.01010761])], [1, 3, array([-2.0265362 ,  1.17002399, -6.03032284])], [34, 4, array([0., 0., 0.])], [34, 5, array([ 0.        ,  0.        , -4.02021523])]]


In [6]:
# crystal structure data
from ase import Atom, Atoms
from ase.io import read, write
from ase.neighborlist import neighbor_list
from ase.visualize.plot import plot_atoms
#from ase_notebook import AseView, ViewConfig, get_example_atoms

palette = ['#43AA8B', '#F8961E', '#F94144']
sub = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

# data pre-processing and visualization
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

path = "absorption_mp_data.pkl"
the_data = pd.read_pickle(path)
the_data.head()
#energy_0 = the_data["energy"][0]
#energy_0

,id,formula,structure,energies,absorption_coefficient,imag_dielectric,real_dielectric,energy_max,bandgap
0,mp-546266,DyBi2IO4,"(Atom('Dy', [0.0, 0.0, 0.0], index=0), Atom('B...","[0.0, 0.0277, 0.0554, 0.0831, 0.1109, 0.1386, ...","[0.0, 0.8362304735329499, 3.309178082718349, 7...","[0.0, 0.0015666666666666665, 0.0031, 0.0046666...","[6.9163, 6.916533333333334, 6.9172, 6.9182, 6....",55.4328,1.3818000000000001
1,mp-9583,K2ZnF4,"(Atom('K', [1.2531985199117757, 1.368027219932...","[0.0, 0.0399, 0.0798, 0.1197, 0.1596, 0.1995, ...","[0.0, 0.1362578822994739, 0.4541860898852432, ...","[0.0, 0.0001, 0.00016666666666666666, 0.0002, ...","[2.2021333333333333, 2.202166666666667, 2.2022...",79.7804,4.3866
2,mp-22988,CsGeCl3,"(Atom('Cs', [7.78333021211009, 0.0, 5.48124386...","[0.0, 0.029, 0.058, 0.087, 0.116, 0.1449, 0.17...","[0.0, 0.515935972729374, 2.0145311056971553, 4...","[0.0, 0.0007, 0.0013666666666666669, 0.0020333...","[3.975733333333333, 3.9758333333333336, 3.9761...",57.9759,2.178
3,mp-861502,AcFeO3,"(Atom('Ac', [0.0, 0.0, 0.0], index=0), Atom('F...","[0.0, 0.0587, 0.1174, 0.176, 0.2347, 0.2934, 0...","[0.0, 11295.053197761259, 44864.495977063896, ...","[0.0, 30.1914, 49.9196, 37.98413333333333, 21....","[67.54320000000001, 59.6215, 29.6067, 1.281, -...",117.3625,0.0
4,mp-1025029,PrHSe,"(Atom('Pr', [-2.0265362, -1.17002398670996, -6...","[0.0, 0.0365, 0.073, 0.1095, 0.1459, 0.1824, 0...","[0.0, 1.6849892209331498, 6.697792903781345, 1...","[0.0, 0.0026999999999999997, 0.005366666666666...","[8.784633333333334, 8.7851, 8.786566666666667,...",72.9718,1.9455999999999998


In [24]:
planck = 6.626076e-34
reduced_planck = planck/(2*np.pi)
reduced_planck

1.0545727487025736e-34

In [ ]:
energy_0 = the_data["energy"][0]
real_dielectric_0 = the_data["real_dielectric"][0]
imag_dielectric_0 = the_data["imag_dielectric"][0]
absorption_0 = the_data["absorption_coefficient"][0]

# Get angular frequncy
ang_freq = []
for i in energy_0:
    angular_frequency = i/reduced_planck
    ang_freq.append(angular_frequency)

#the_data["angular_frequency"][0] = ang_freq


In [47]:
#the_data["energies"]
# energy_0 = np.size(the_data["energies"][0])
#abs_cof = np.size(the_data["absorption_coefficient"][0])
#ima_diel = np.size(the_data["imag_dielectric"][0])
rel_diel = np.size(the_data["real_dielectric"][0])

'''energy_0 = the_data["energies"][0]
i = 0
for x in energy_0:
    i = i +1

print(i)'''
rel_diel

2001